In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms, models 
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import copy

In [2]:
mean = np.array([.5,.5,.5])
std = np.array([.25,.25,.25])

In [4]:
data_transforms = {
    'train':transforms.Compose([
        transforms.RandomSizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val':transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
}

In [11]:
data_dir = 'data/hymenoptera_data'
classes = ['train', 'val']

In [12]:
image_datasets = {x:datasets.ImageFolder(os.path.join(data_dir,x), data_transforms[x]) for x in classes}

In [13]:
dataloaders = {x:torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True) for x in classes}

In [14]:
data_sizes = {x: len(image_datasets[x]) for x in classes}

In [21]:
class_names = image_datasets[classes[0]].classes

In [27]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict)
    best_acc = .0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        for phase in classes:
            if phase == classes[0]:
                model.train()
            else:
                model.eval()
                
            running_loss = .0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                
                with torch.set_grad_enabled(phase==classes[0]):
                    outputs = model(inputs)
                    _,preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == classes[0]:
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == classes[0]:
                scheduler.step()
                
            epoch_loss = running_loss / data_sizes[phase]
            epoch_acc = running_corrects.double() / data_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            if phase == classes[1] and epoch_acc>best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
        
        time_elapsed = time.time() - since
        
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('Best val Acc: {:4f}'.format(best_acc))
        
        model.load_state_dict(best_model_wts)
        return model

In [29]:
model = models.resnet18(pretrained=True)
for params in model.parameters():
    params.require_grad = False

In [25]:
num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=.1)

In [30]:
model = train_model(model, criterion, optimizer,scheduler=step_lr_scheduler, num_epochs=20)

Epoch 0/19
----------
train Loss: 8.9772 Acc: 0.0000
val Loss: 10.6967 Acc: 0.0000

Training complete in 1m 39s
Best val Acc: 0.000000


AttributeError: 'function' object has no attribute 'copy'